In [1]:
import pandas as pd
import json
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException, WebDriverException, ElementClickInterceptedException, InvalidElementStateException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from datetime import datetime
from selenium import webdriver
import time

In [2]:
df_tags=pd.read_excel('tags.xlsx')
df_tags

,id,text
0,1,desenvolvimento de pessoas
1,2,política nacional de desenvolvimento de pessoas
2,3,PNDP
3,4,inovação
4,5,melhoria
5,6,ações implementadas
6,7,publicidade
7,8,capacitação
8,9,cursos
9,10,conhecimento


In [26]:
json_data=df_tags.to_json(orient='records',force_ascii=False)

In [27]:
# Salva o objeto JSON em um arquivo
with open('tags.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

<h1>Baixando tags do portal</h1>

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.81M/6.81M [00:01<00:00, 5.34MB/s]
C:\Users\Alexandre Cunha\AppData\Local\Temp\ipykernel_18660\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
url='https://www.gov.br/servidor/pt-br/observatorio-de-pessoal-govbr/@@template_listagem_produto_dados?form.submitted=1&texto_busca_rapida=&filtro_tema=&tema='

In [7]:
tags=driver.find_elements(By.CLASS_NAME, 'tags')

In [8]:
lista_tags=[]
for tag in tags:
    links=tag.find_elements(By.CLASS_NAME, 'link-produtos')
    for link_produto in links:
        lista_tags.append(link_produto.text)

In [9]:
len(lista_tags)

424

In [10]:
df_tags=pd.DataFrame()
df_tags['TAGS']=lista_tags
df_tags=df_tags.drop_duplicates()
df_tags=df_tags.reset_index()
df_tags=df_tags.drop('index', axis=1)
df_tags=df_tags.reset_index()
df_tags=df_tags.rename({'index':'id','TAGS':'text'}, axis=1)
df_tags

,id,text
0,0,afastamentos
1,1,saúde
2,2,Painel Estatístico de Pessoal
3,3,PEP
4,4,Servidores
...,...,...
291,291,Nome servidor
292,292,função
293,293,nome do órgão
294,294,data início exercício


In [12]:
df_tags=df_tags.drop_duplicates()

<h1>Testando Sinonimos das palavras</h1>

In [17]:
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to C:\Users\Alexandre
[nltk_data]     Cunha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Alexandre
[nltk_data]     Cunha\AppData\Roaming\nltk_data...


True

In [18]:
# Define a palavra de referência
palavra_referencia = "melhoria"
# Busca os synsets (conjuntos de sinônimos) da palavra de referência
synsets = wordnet.synsets(palavra_referencia, lang="por")

In [19]:
# Lista os sinônimos de cada synset encontrado
sinonimos = set()
for synset in synsets:
    for lemma in synset.lemmas(lang="por"):
        sinonimos.add(lemma.name())

In [20]:
# Imprime a lista de sinônimos encontrados
print("Sinônimos de {}: {}".format(palavra_referencia, list(sinonimos)))

Sinônimos de melhoria: ['progresso', 'melhoramento', 'melhoria', 'aprimoramento']


In [21]:
tag=[]
sinonimo=[]
for index, row in df_tags.iterrows():
    synsets = wordnet.synsets(row['text'], lang="por")
    for synset in synsets:
        for lemma in synset.lemmas(lang="por"):
            tag.append(row['text'])
            sinonimo.append(lemma.name())

In [22]:
len(tag)

906

In [25]:
df_tags_sinonimos=pd.DataFrame()
df_tags_sinonimos['TAGS']=tag
df_tags_sinonimos['Sinonimos']=sinonimo
df_tags_sinonimos=df_tags_sinonimos.drop_duplicates()
df_tags_sinonimos

,TAGS,Sinonimos
0,saúde,saúde
1,saúde,bem-estar
2,saúde,eudemonismo
5,Servidores,Servidores
6,Servidores,servidor
...,...,...
896,função,Papel_social
899,função,propósito
900,função,Sub-rotina
902,função,sub-rotina


In [27]:
df_tags_sinonimos.nunique()

TAGS          83
Sinonimos    399
dtype: int64

In [29]:
df_tags_sinonimos2=df_tags.copy()
df_tags_sinonimos2=df_tags_sinonimos2.rename({'text':'TAGS'}, axis=1)
df_tags_sinonimos2['Sinonimos']=df_tags_sinonimos2['TAGS']
df_tags_sinonimos2=df_tags_sinonimos2[['TAGS','Sinonimos']]
df_tags_sinonimos2

,TAGS,Sinonimos
0,afastamentos,afastamentos
1,saúde,saúde
2,Painel Estatístico de Pessoal,Painel Estatístico de Pessoal
3,PEP,PEP
4,Servidores,Servidores
...,...,...
291,Nome servidor,Nome servidor
292,função,função
293,nome do órgão,nome do órgão
294,data início exercício,data início exercício


In [30]:
df_tags_sinonimos_rev1=pd.concat([df_tags_sinonimos,df_tags_sinonimos2])
df_tags_sinonimos_rev1=df_tags_sinonimos_rev1.drop_duplicates()
df_tags_sinonimos_rev1.nunique()

TAGS         296
Sinonimos    636
dtype: int64

In [31]:
df_tags_sinonimos_rev1

,TAGS,Sinonimos
0,saúde,saúde
1,saúde,bem-estar
2,saúde,eudemonismo
5,Servidores,Servidores
6,Servidores,servidor
...,...,...
290,Pessoas politicamente expostas,Pessoas politicamente expostas
291,Nome servidor,Nome servidor
293,nome do órgão,nome do órgão
294,data início exercício,data início exercício
